In [6]:
from datetime import datetime, date, timedelta, time

path = "./logs/motion_frames/"
timestamp = str(datetime.now())[:19].replace(":",";").replace(" ", "_")
print(timestamp)

2024-02-18_17;05;08
